In [2]:
!pip install mne -q

In [8]:
from glob import glob
import scipy.io
import torch.nn as nn
import torch
import numpy as np
import mne

In [3]:
dataset_path = "/kaggle/input/eeg-idd-data/EEG_Dataset/EEG dataset of individuals with intellectual and developmental disorder and healthy controls while observing rest and musical stimulus/Data"

In [19]:
IDD_data_path = '/kaggle/input/eeg-idd-data/EEG_Dataset/EEG dataset of individuals with intellectual and developmental disorder and healthy controls while observing rest and musical stimulus/Data/CleanData/CleanData_IDD/Rest'
TDC_data_path = '/kaggle/input/eeg-idd-data/EEG_Dataset/EEG dataset of individuals with intellectual and developmental disorder and healthy controls while observing rest and musical stimulus/Data/CleanData/CleanData_TDC/Rest'

In [20]:
print(IDD_data_path)

/kaggle/input/eeg-idd-data/EEG_Dataset/EEG dataset of individuals with intellectual and developmental disorder and healthy controls while observing rest and musical stimulus/Data/CleanData/CleanData_IDD/Rest


In [21]:
def convertmat2mne(data):
    ch_names = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
    ch_types = ['eeg'] * 14
    sampling_freq=128
    info = mne.create_info(ch_names, ch_types=ch_types, sfreq=sampling_freq)
    info.set_montage('standard_1020')
    data=mne.io.RawArray(data, info)
    data.set_eeg_reference()
    data.filter(l_freq=1,h_freq=30)
    epochs=mne.make_fixed_length_epochs(data,duration=4,overlap=0)
    return epochs.get_data()

In [22]:
idd_subject=[]
for idd in glob(IDD_data_path+'/*.mat'):
    print(idd)
    data=scipy.io.loadmat(idd)['clean_data']
    data=convertmat2mne(data)
    idd_subject.append(data)

/kaggle/input/eeg-idd-data/EEG_Dataset/EEG dataset of individuals with intellectual and developmental disorder and healthy controls while observing rest and musical stimulus/Data/CleanData/CleanData_IDD/Rest/NDS003_Rest_CD.mat
Creating RawArray with float64 data, n_channels=14, n_times=15360
    Range : 0 ... 15359 =      0.000 ...   119.992 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter leng

In [23]:
%%capture
tdc_subject=[]
for tdc in glob(TDC_data_path+'/*.mat'):
    data=scipy.io.loadmat(tdc)['clean_data']
    data=convertmat2mne(data)
    tdc_subject.append(data)

In [24]:
control_epochs_labels=[len(i)*[0] for i in tdc_subject]
patients_epochs_labels=[len(i)*[1] for i in idd_subject]
print(len(control_epochs_labels),len(patients_epochs_labels))

7 7


In [25]:
data_list=tdc_subject+idd_subject
label_list=control_epochs_labels+patients_epochs_labels
groups_list=[[i]*len(j) for i, j in enumerate(data_list)]
print(len(data_list),len(label_list),len(groups_list))

14 14 14


In [26]:
from sklearn.model_selection import GroupKFold,LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
gkf=GroupKFold()
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.preprocessing import StandardScaler
#https://stackoverflow.com/questions/50125844/how-to-standard-scale-a-3d-matrix
class StandardScaler3D(BaseEstimator,TransformerMixin):
    #batch, sequence, channels
    def __init__(self):
        self.scaler = StandardScaler()

    def fit(self,X,y=None):
        self.scaler.fit(X.reshape(-1, X.shape[2]))
        return self

    def transform(self,X):
        return self.scaler.transform(X.reshape( -1,X.shape[2])).reshape(X.shape)

In [27]:
import numpy as np
data_array=np.concatenate(data_list)
label_array=np.concatenate(label_list)
group_array=np.concatenate(groups_list)
data_array=np.moveaxis(data_array,1,2)

print(data_array.shape,label_array.shape,group_array.shape)

(420, 512, 14) (420,) (420,)


In [28]:
accuracy=[]
for train_index, val_index in gkf.split(data_array, label_array, groups=group_array):
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    scaler=StandardScaler3D()
    train_features=scaler.fit_transform(train_features)
    val_features=scaler.transform(val_features)
   

    break
train_features.shape 

(330, 512, 14)

In [29]:
from tensorflow.keras.layers import Input,Dense,concatenate,Flatten,GRU,Conv1D
from tensorflow.keras.models import Model
#resource:https://github.com/dll-ncai/eeg_pre-diagnostic_screening/blob/master/code/chrononet/chrono.py

def block(input):
    conv1 = Conv1D(32, 2, strides=2,activation='relu',padding="same")(input)
    conv2 = Conv1D(32, 4, strides=2,activation='relu',padding="causal")(input)
    conv3 = Conv1D(32, 8, strides=2,activation='relu',padding="causal")(input)
    x = concatenate([conv1,conv2,conv3],axis=2)
    return x


2024-02-28 21:36:05.260324: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 21:36:05.260430: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 21:36:05.380947: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [30]:
input = Input(shape=(512,14))
block1=block(input)
block2=block(block1)
block3=block(block2)

gru_out1 = GRU(32,activation='tanh',return_sequences=True)(block3)
gru_out2 = GRU(32,activation='tanh',return_sequences=True)(gru_out1)
gru_out = concatenate([gru_out1,gru_out2],axis=2)
gru_out3 = GRU(32,activation='tanh',return_sequences=True)(gru_out)
gru_out = concatenate([gru_out1,gru_out2,gru_out3])
gru_out4 = GRU(32,activation='tanh')(gru_out)

In [31]:
predictions = Dense(1,activation='sigmoid')(gru_out4)
model = Model(inputs=input, outputs=predictions)

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [32]:
model.fit(train_features,train_labels,epochs=10,batch_size=128,validation_data=(val_features,val_labels))

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 575ms/step - accuracy: 0.5590 - loss: 0.6693 - val_accuracy: 0.9000 - val_loss: 0.6309
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.9389 - loss: 0.5765 - val_accuracy: 0.8889 - val_loss: 0.5458
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.9617 - loss: 0.4233 - val_accuracy: 1.0000 - val_loss: 0.3164
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.9856 - loss: 0.2304 - val_accuracy: 1.0000 - val_loss: 0.1339
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.9891 - loss: 0.0915 - val_accuracy: 0.9778 - val_loss: 0.1150
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.9931 - loss: 0.0391 - val_accuracy: 1.0000 - val_loss: 0.0138
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 1.0000 - loss: 0.0153 - val_accuracy: 0.9889 - val_loss: 0.0448
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 1.0000 - loss: 0.0067 - val_accuracy: 0.9889 - val_loss: 0.070